<a href="https://colab.research.google.com/github/vikysena9-design/python-ai-Rudnikova-Viktoria/blob/main/notebooks/week2b_read_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Week 2: Data Analysis — Чтение и проверка данных

**Цель**: Научиться читать CSV-файлы из репозитория GitHub в Google Colab и выполнять базовую проверку данных с помощью pandas.

**Данные:**
- `horses.csv` — информация о лошадях: клички, годы жизни, пол, порода

**Что мы делаем:**
1. Клонируем ваш репозиторий GitHub в Colab
2. Читаем CSV-файл с данными о лошадях в pandas DataFrame
3. Очищаем и анализируем структуру данных
4. Выполняем быструю валидацию: проверяем количество записей, типы данных, пропуски

## 🐱 [1] Клонируем репозиторий курса в Colab

In [2]:
# 🐴 Шаг 1. Клонируем ваш репозиторий в Colab

import os

if not os.path.exists("python-ai-Rudnikova-Viktoria"):
    !git clone -q https://github.com/vikysena9-design/python-ai-Rudnikova-Viktoria.git

%cd python-ai-Rudnikova-Viktoria

print("✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Rudnikova-Viktoria")

/content/python-ai-Rudnikova-Viktoria/python-ai-Rudnikova-Viktoria
✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Rudnikova-Viktoria


## 📥 [2A] Простое чтение CSV-файлов в pandas

Сначала просто прочитаем оба CSV-файла в объекты `DataFrame`, без каких‑либо изменений.

После этого мы узнаем, сколько строк загружено в каждый датасет.

In [8]:
# 🐴 Шаг 2. Чтение CSV-файла с данными о лошадях

import pandas as pd

df_horses = pd.read_csv("data/horses.csv")

print("✅ Загружено строк в датасет:", len(df_horses))
print("\n📋 Первые 3 строки данных:")
print(df_horses.head(3))
print("\n🔍 Структура данных:")
print(df_horses.info())

✅ Загружено строк в датасет: 12309

📋 Первые 3 строки данных:
                                     horse        horseLabel  birthYear  \
0  http://www.wikidata.org/entity/Q5172152  Coronado's Quest     1995.0   
1  http://www.wikidata.org/entity/Q5180163           Cozzene     1980.0   
2  http://www.wikidata.org/entity/Q5190874   Cryptoclearance     1984.0   

   deathYear genderLabel             breedLabel  
0     2006.0       самец  чистокровная верховая  
1     2008.0       самец  чистокровная верховая  
2     2009.0       самец  чистокровная верховая  

🔍 Структура данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12309 entries, 0 to 12308
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   horse        12309 non-null  object 
 1   horseLabel   12309 non-null  object 
 2   birthYear    12308 non-null  float64
 3   deathYear    1581 non-null   float64
 4   genderLabel  11280 non-null  object 
 5   breedLabe

## 🧹 [2B] Очистка и переименование столбцов

В исходном CSV-файле есть **технические столбцы**, которые полезны для Викиданных, но мешают простому анализу:

- Столбец `horse` с URL (ссылкой на объект Wikidata) — нам не нужна ссылка, нам достаточно клички лошади.
- Столбцы `horseLabel`, `genderLabel`, `breedLabel` содержат читаемые подписи (кличка, пол, порода).

В этом шаге мы:
- удалим столбец с URL Wikidata (`horse`);
- переименуем `horseLabel → horse`, `genderLabel → gender`, `breedLabel → breed`;
- приведём числовые столбцы (`birthYear`, `deathYear`) к типу `int`.

При приведении к числам мы используем:

- `pd.to_numeric(..., errors="coerce")` — преобразует значения в числа, некорректные значения превращает в `NaN`;
- `fillna(0)` — заменяет пропущенные значения (`NaN`) на 0;
- `astype(int)` — переводит столбец к целочисленному типу.

> ⚠️ **Важно:** если в ваших данных есть столбцы с URL Wikidata и столбцы вида `*Label`, этот шаг обязателен, чтобы получить аккуратные таблички для анализа.

In [7]:
# 🧹 Шаг 2B. Очистка и переименование столбцов (безопасная версия)

# Проверяем, есть ли исходные столбцы Wikidata — если да, выполняем очистку
if "horse" in df_horses.columns and "horseLabel" in df_horses.columns:
    # Удаляем только столбец с URL (технический идентификатор)
    df_horses = df_horses.drop(columns=["horse"])

    # Переименовываем *Label-столбцы только если они существуют
    rename_mapping = {}
    if "horseLabel" in df_horses.columns:
        rename_mapping["horseLabel"] = "horse"
    if "genderLabel" in df_horses.columns:
        rename_mapping["genderLabel"] = "gender"
    if "breedLabel" in df_horses.columns:
        rename_mapping["breedLabel"] = "breed"

    if rename_mapping:
        df_horses = df_horses.rename(columns=rename_mapping)

    # Приводим числовые столбцы к целочисленному типу
    for col in ["birthYear", "deathYear"]:
        if col in df_horses.columns:
            df_horses[col] = pd.to_numeric(
                df_horses[col], errors="coerce"
            ).fillna(0).astype(int)

    print("✅ Данные успешно очищены и подготовлены к анализу")

elif "horse" in df_horses.columns and "horseLabel" not in df_horses.columns:
    print("ℹ️ Данные уже очищены ранее — столбцы *Label отсутствуют")

else:
    print("⚠️ Неожиданная структура данных: проверьте наличие столбцов horse/horseLabel")

print("\n📋 Структура датасета после обработки:")
print(df_horses.info())
print("\n🔍 Пример данных:")
print(df_horses.head(3))

⚠️ Неожиданная структура данных: проверьте наличие столбцов horse/horseLabel

📋 Структура датасета после обработки:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12309 entries, 0 to 12308
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   birthYear  12309 non-null  int64 
 1   deathYear  12309 non-null  int64 
 2   gender     11280 non-null  object
 3   breed      12309 non-null  object
dtypes: int64(2), object(2)
memory usage: 384.8+ KB
None

🔍 Пример данных:
   birthYear  deathYear gender                  breed
0       1995       2006  самец  чистокровная верховая
1       1980       2008  самец  чистокровная верховая
2       1984       2009  самец  чистокровная верховая


## 🔍 [3] Обзор данных: структура и первые строки

Сделаем короткий обзор обоих DataFrame:

- посмотрим размер таблицы (`shape`);
- выведем список столбцов;
- посмотрим первые несколько строк;
- дополнительно посчитаем базовую статистику по бюджету (`capital_cost`).

Для удобства напишем маленькую функцию `show_info(df, name)`, чтобы не повторять один и тот же код два раза.

In [ ]:
def show_info(df, name, n=5):
    """Краткий обзор DataFrame: имя, размер, список столбцов и первые строки."""
    print(f"\n📊 {name}")
    print("Размер:", df.shape)
    print("Столбцы:", ", ".join(df.columns))
    print("\nПервые строки:")
    print(df.head(n))

# 🔍 Шаг 3. Обзор данных

show_info(df_cost, "Бюджеты мультфильмов (df_cost)")
show_info(df_genre, "Жанры, страны и продолжительность (df_genre)")

print("\n📈 Статистика по бюджету (capital_cost):")
print(df_cost["capital_cost"].describe())

In [ ]:
# То же самое, но в миллионах долларов (млн $)
stats_millions = (df_cost["capital_cost"] / 1e6).describe().round(2)
print("\n📈 Статистика по бюджету в млн $:")
print(stats_millions)

## ✅ [4] Быстрая проверка и валидация данных

Здесь мы посмотрим:

- сколько **уникальных** фильмов, стран и жанров есть в данных;
- **какие страны встречаются чаще всего** (Топ‑5 по числу строк);
- **какие жанры самые популярные** (Топ‑10 по числу строк).

Функция `value_counts()`:
- считает, сколько раз каждое значение встречается в столбце;
- сортирует результаты по убыванию.

Метод `.head()` берёт первые N строк, поэтому  
`df_genre["country"].value_counts().head()` даёт **Топ‑5 стран по числу записей**.

In [ ]:
# ✅ Шаг 4. Быстрая проверка и валидация данных

print("🔍 Быстрая проверка данных")

# Датасет 1: бюджеты
print("\nУникальных мультфильмов в df_cost:", df_cost["film"].nunique())
print("Диапазон бюджетов (млн $):",
      df_cost["capital_cost"].min() / 1e6, "—", df_cost["capital_cost"].max() / 1e6)

# Датасет 2: жанры, страны, длительность
print("\nУникальных мультфильмов в df_genre:", df_genre["film"].nunique())
print("Уникальных стран:", df_genre["country"].nunique())
print("Уникальных жанров:", df_genre["genre"].nunique())

print("\nТоп-5 стран по числу записей:")
print(df_genre["country"].value_counts().head())

print("\nТоп-10 жанров:")
print(df_genre["genre"].value_counts().head(10))

## 📝 Summary

**Что мы сделали в этом ноутбуке (Week 2):**

- ✅ Клонировали репозиторий GitHub в Colab
- ✅ Прочитали 2 CSV-файла из `data/examples/`
- ✅ Удалили URL Wikidata и переименовали столбцы (`*Label → короткие имена`)
- ✅ Проверили структуру данных (размер, столбцы, первые строки)
- ✅ Посмотрели базовую статистику по бюджету (`capital_cost`)
- ✅ Выполнили быструю валидацию:
  - количество уникальных фильмов, стран, жанров
  - диапазоны значений
  - топ стран и жанров по числу записей

Теперь у нас есть **аккуратные, проверенные таблицы**, с которыми удобно работать дальше.

В отдельном ноутбуке для следующей недели мы будем использовать **те же данные** для:
- более сложного анализа (группировки, фильтрация),
- и построения визуализаций (графики и диаграммы). 🎨